In [ ]:
# End-to-End Revenue Analysis

This notebook performs a comprehensive analysis of the relationship between Time on Page (TOP) and Revenue.

**Outputs:**
- `figs/fig1_hexbin.png` - Hexbin plot with LOWESS trend
- `figs/fig2_spline_uncontrolled.png` - Spline regression (uncontrolled)
- `figs/fig3_marginal_effects.png` - Controlled marginal effects
- `report/data_profile.json` - Data profiling and key statistics

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.api import add_constant
from statsmodels.regression.linear_model import OLS
from statsmodels.nonparametric.smoothers_lowess import lowess
from patsy import dmatrix, build_design_matrices

print("All libraries imported successfully!")

In [ ]:
# Configuration and Setup
# Adapted for the current workspace structure

# Update paths to match your workspace structure
DATA_PATH = "../data/testdata.csv"  # Relative to analysis folder
OUT_DIR = ".."  # Root of the project
FIG_DIR = os.path.join(OUT_DIR, "figs")
REP_DIR = os.path.join(OUT_DIR, "report")

# Create output directories if they don't exist
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(REP_DIR, exist_ok=True)

print(f"Data path: {DATA_PATH}")
print(f"Figures will be saved to: {FIG_DIR}")
print(f"Reports will be saved to: {REP_DIR}")
print(f"Data file exists: {os.path.exists(DATA_PATH)}")

## 1. Data Loading and Cleaning

In [ ]:
# Load and clean the data
raw = pd.read_csv(DATA_PATH)
print(f"Loaded {len(raw)} rows with {len(raw.columns)} columns")
print("Columns:", list(raw.columns))

initial_rows = len(raw)
missing_counts = raw.isna().sum().to_dict()
print(f"\nMissing values:\n{pd.Series(missing_counts)}")

# Drop rows with missing revenue or top values
df = raw.dropna(subset=["revenue", "top"]).copy()
after_drop_rows = len(df)
print(f"\nAfter dropping missing revenue/top: {after_drop_rows} rows ({initial_rows - after_drop_rows} dropped)")

# Cast categorical variables
for col in ["browser", "platform", "site"]:
    if col in df.columns:
        df[col] = df[col].astype("category")
        print(f"Cast {col} as category with {len(df[col].cat.categories)} levels")

# Display basic info
print(f"\nDataset shape: {df.shape}")
print(f"Data types:\n{df.dtypes}")

In [ ]:
# Winsorize outliers at 1%/99% percentiles
winsor_thresholds = {}
rows_winsorized = {}

for col in ["revenue", "top"]:
    # Calculate percentiles
    lo, hi = df[col].quantile([0.01, 0.99])
    winsor_thresholds[col] = (float(lo), float(hi))
    
    # Count rows that will be affected
    rows_winsorized[col] = int(((df[col] < lo) | (df[col] > hi)).sum())
    
    # Apply winsorization
    df[col] = np.clip(df[col], lo, hi)
    
    print(f"{col}: winsorized {rows_winsorized[col]} rows at [{lo:.2f}, {hi:.2f}]")

# Create log-transformed revenue variable
df["log_rev"] = np.log1p(df["revenue"])
print(f"\nCreated log_rev variable (log(1 + revenue))")

# Show summary statistics
print(f"\nSummary statistics after cleaning:")
print(df[["revenue", "top", "log_rev"]].describe())

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Calculate EDA statistics
eda_stats = {
    "revenue": {
        "mean": float(df["revenue"].mean()),
        "median": float(df["revenue"].median()),
        "sd": float(df["revenue"].std()),
    },
    "top": {
        "mean": float(df["top"].mean()),
        "median": float(df["top"].median()),
        "sd": float(df["top"].std()),
    },
    "corr_revenue_top": float(df[["revenue","top"]].corr().iloc[0,1])
}

print("EDA Statistics:")
for var, stats in eda_stats.items():
    if isinstance(stats, dict):
        print(f"{var}:")
        for stat, val in stats.items():
            print(f"  {stat}: {val:.3f}")
    else:
        print(f"{var}: {stats:.3f}")

# Generate Figure 1: Hexbin plot with LOWESS trend
plt.figure(figsize=(7,5), dpi=150)
hb = plt.hexbin(df["top"], df["revenue"], gridsize=40, mincnt=1)
plt.colorbar(hb, label='Count')

# Add LOWESS smoothing trend
low = lowess(df["revenue"], df["top"], frac=0.2, return_sorted=True)
plt.plot(low[:,0], low[:,1], 'r-', linewidth=2, label='LOWESS trend')

plt.xlabel("Time on Page (TOP)")
plt.ylabel("Revenue")
plt.title("Revenue vs Time on Page (EDA)")
plt.legend()
plt.tight_layout()

# Save the figure
fig1_path = os.path.join(FIG_DIR, "fig1_hexbin.png")
plt.savefig(fig1_path)
print(f"\nSaved Figure 1: {fig1_path}")
plt.show()

## 3. Statistical Modeling

In [ ]:
# Helper function for OLS regression with robust standard errors
def fit_ols(y, X):
    """Fit OLS model with robust (HC1) standard errors"""
    Xc = add_constant(X, has_constant="add")
    model = OLS(y, Xc).fit(cov_type="HC1")  # robust SEs
    return model

# Model 1: revenue ~ top (simple linear relationship)
print("Model 1: revenue ~ top")
m1 = fit_ols(df["revenue"], df[["top"]])
print(m1.summary())
print(f"\nModel 1 R-squared: {m1.rsquared:.4f}")
print(f"TOP coefficient: {m1.params.get('top', float('nan')):.4f}")

In [ ]:
# Model 2: log1p(revenue) ~ spline(top, df=4) - Check for diminishing returns
print("Model 2: log1p(revenue) ~ spline(top, df=4)")
X_spline = dmatrix("bs(top, df=4, degree=3, include_intercept=False)", {"top": df["top"]}, return_type="dataframe")
m2 = fit_ols(df["log_rev"], X_spline)
print(f"Model 2 R-squared: {m2.rsquared:.4f}")

# Generate predictions for plotting
top_grid = np.linspace(df["top"].min(), df["top"].max(), 200)
Xgrid_spline = dmatrix("bs(top, df=4, degree=3, include_intercept=False)", {"top": top_grid}, return_type="dataframe")
Xgrid_spline_c = add_constant(Xgrid_spline, has_constant="add")
pred2 = m2.get_prediction(Xgrid_spline_c)
pred2_mean = pred2.predicted_mean
pred2_ci = pred2.conf_int()

# Generate Figure 2: Spline regression (uncontrolled)
plt.figure(figsize=(7,5), dpi=150)
plt.plot(top_grid, pred2_mean, 'b-', linewidth=2, label='Spline fit')
plt.fill_between(top_grid, pred2_ci[:,0], pred2_ci[:,1], alpha=0.2, color='blue', label='95% CI')
plt.xlabel("Time on Page (TOP)")
plt.ylabel("log(1 + Revenue)")
plt.title("Diminishing Returns Check (Spline, Uncontrolled)")
plt.legend()
plt.tight_layout()

# Save the figure
fig2_path = os.path.join(FIG_DIR, "fig2_spline_uncontrolled.png")
plt.savefig(fig2_path)
print(f"\nSaved Figure 2: {fig2_path}")
plt.show()

In [ ]:
# Model 3: log1p(revenue) ~ spline(top) + C(browser) + C(platform) + C(site)
# This controls for other factors while examining the TOP effect
print("Model 3: log1p(revenue) ~ spline(top) + controls")

design = dmatrix(
    "bs(top, df=4, degree=3, include_intercept=False) + C(browser) + C(platform) + C(site)",
    df,
    return_type="dataframe"
)

design_info = design.design_info
m3 = fit_ols(df["log_rev"], design)
print(f"Model 3 R-squared: {m3.rsquared:.4f}")
print(f"Number of parameters: {len(m3.params)}")

# Show a summary of the controlled model
print(f"\nControlled model summary:")
print(f"AIC: {m3.aic:.2f}")
print(f"BIC: {m3.bic:.2f}")

In [ ]:
# Marginal Effects Analysis
# Compare predictions at different TOP values while holding other factors constant

def predict_logrev(model, top_values, df_ref):
    """Predict log(revenue) for given TOP values, holding other factors at reference levels"""
    # Build a design matrix holding other factors at their reference (first category)
    tmp = df_ref.copy()
    tmp = tmp.iloc[:1].copy()  # single row template
    tmp = pd.concat([tmp]*len(top_values), ignore_index=True)
    tmp["top"] = top_values
    
    # Set reference categories to the first levels
    for col in ["browser","platform","site"]:
        if col in tmp.columns and str(tmp[col].dtype) == "category":
            tmp[col] = tmp[col].cat.categories[0]
    
    X = build_design_matrices([design_info], tmp)[0]
    Xc = add_constant(X, has_constant="add")
    pred = model.get_prediction(Xc).predicted_mean
    return pred

# Generate predictions across TOP range
top_vals = np.linspace(df["top"].quantile(0.05), df["top"].quantile(0.95), 120)
log_pred = predict_logrev(m3, top_vals, df)

# Generate Figure 3: Controlled marginal effects
plt.figure(figsize=(7,5), dpi=150)
plt.plot(top_vals, np.expm1(log_pred), 'g-', linewidth=2)
plt.xlabel("Time on Page (TOP)")
plt.ylabel("Predicted Revenue (Controlled)")
plt.title("Controlled Marginal Effect of TOP")
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save the figure
fig3_path = os.path.join(FIG_DIR, "fig3_marginal_effects.png")
plt.savefig(fig3_path)
print(f"Saved Figure 3: {fig3_path}")
plt.show()

In [ ]:
# Calculate key effect size: % change from mean TOP to mean + 1 SD
mu = df["top"].mean()
sd = df["top"].std()

log_mu = predict_logrev(m3, np.array([mu]), df)[0]
log_mu_sd = predict_logrev(m3, np.array([mu+sd]), df)[0]
percent_change_mu_to_mu_sd = float(np.expm1(log_mu_sd - log_mu) * 100.0)

print(f"Key Effect Size:")
print(f"Mean TOP: {mu:.2f}")
print(f"Mean + 1 SD TOP: {mu + sd:.2f}")
print(f"Predicted revenue increase from mean to mean+1SD: {percent_change_mu_to_mu_sd:.2f}%")

## 4. Final Report Generation

In [ ]:
# Generate comprehensive data profile and save results
profile = {
    "initial_rows": initial_rows,
    "rows_after_drop": after_drop_rows,
    "n_columns": raw.shape[1],
    "missing_counts": missing_counts,
    "winsorized_thresholds": winsor_thresholds,
    "rows_winsorized": rows_winsorized,
    "eda_stats": eda_stats,
    "m1_slope_top": float(m1.params.get("top", float("nan"))),
    "m1_r2": float(m1.rsquared),
    "m2_r2": float(m2.rsquared),
    "m3_r2": float(m3.rsquared),
    "controlled_effect_mu_to_mu_plus_1sd_percent": percent_change_mu_to_mu_sd
}

# Save the profile to JSON
profile_path = os.path.join(REP_DIR, "data_profile.json")
with open(profile_path, "w") as f:
    json.dump(profile, f, indent=2)

print("="*60)
print("ANALYSIS COMPLETE!")
print("="*60)
print(f"Data profile saved: {profile_path}")
print(f"Figures saved:")
print(f"  - {os.path.join(FIG_DIR, 'fig1_hexbin.png')}")
print(f"  - {os.path.join(FIG_DIR, 'fig2_spline_uncontrolled.png')}")
print(f"  - {os.path.join(FIG_DIR, 'fig3_marginal_effects.png')}")
print(f"\nKey Finding:")
print(f"A 1 standard deviation increase in Time on Page is associated with a")
print(f"{percent_change_mu_to_mu_sd:.2f}% increase in revenue (controlled for other factors).")